In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import re
import pymysql
import time

lib_url = 'https://mail.sjtu.edu.cn/zimbra/mail#1'
edge_or_not = 0
headless_or_not = 0
if edge_or_not:
    myoption = webdriver.EdgeOptions()
    if headless_or_not:
        myoption.add_argument("--headless")#指定无头模式
    myoption.add_experimental_option('detach', True)
    myoption.add_argument('-ignore-certificate-errors')
    myoption.add_argument('-ignore -ssl-errors')
    # myoption.add_argument("--disable-extensions")
    myoption.add_experimental_option("excludeSwitches", ['enable-automation', 'enable-logging'])
    driver = webdriver.Edge(r"/Users/lzd/Desktop/workspace/driver/chromedriver-mac-x64/chromedriver", options=myoption)  # 创建浏览器实例
else:
    # 启动Chrome
    # driver = webdriver.Chrome()
    # driver.get('http://www.baidu.com/')  # 打开百度
    myoption = webdriver.ChromeOptions()
    if headless_or_not:
        myoption.add_argument("--headless")#指定无头模式
    myoption.add_experimental_option('detach', True)
    myoption.add_argument('-ignore-certificate-errors')
    myoption.add_argument('-ignore -ssl-errors')
    # myoption.add_argument("--disable-extensions")
    myoption.add_experimental_option("excludeSwitches", ['enable-automation', 'enable-logging'])
    driver = webdriver.Chrome(r"D:\vscode_workspace\Python\ZhongYi\chromedriver_win64\chromedriver.exe")# options=myoption)  # 创建浏览器实例
driver.get(lib_url)
driver.maximize_window()
# 创建 WebDriverWait 对象，最大等待时间为 60 秒
wait = WebDriverWait(driver, 60)


C:\Users\15692\AppData\Local\Temp\ipykernel_16204\2386771053.py:33: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"D:\vscode_workspace\Python\ZhongYi\chromedriver_win64\chromedriver.exe")# options=myoption)  # 创建浏览器实例


In [2]:
driver.switch_to.window(driver.window_handles[1])

In [3]:
# 去除 html 关键字，尤其是 <br>
text = []
for i in range(len(contents)):
    tmp_text = contents[i].prettify(formatter=None)
    tmp_text = tmp_text.split('\n')
    text.extend(tmp_text)
i=0
terms = []
tmp = ""
while i<len(text)-1:
    tmp = text[i].replace(" ", "")
    if len(text[i])>=1 and text[i][0]!='<' and text[i][1]!='<':
        terms.append(tmp)
    i = i + 1
# print(terms)
print("数据已转化为列表")

NameError: name 'contents' is not defined

In [4]:
# 条目结构体
class item:
    def __init__(self) -> None:
        self.title = ""
        self.authers = ""
        self.units = ""
        self.journal = ""
        self.keywords = ""
        self.source = ""
        self.abstract = ""
        self.href = ""
        self.quote = -1
        self.ISSN = ""

# 获得数据
soup = BeautifulSoup(driver.page_source, 'html.parser')
contents = soup.find_all("div",{"class":{"MsgBody-html"}})
# print(len(contents))
print("数据已获取")

# 去除 html 关键字，尤其是 <br>
text = []
for i in range(len(contents)):
    tmp_text = contents[i].prettify(formatter=None)
    tmp_text = tmp_text.split('\n')
    text.extend(tmp_text)
i=0
terms = []
tmp = ""
while i<len(text)-1:
    tmp = text[i].replace(" ", "")
    if len(text[i])>=1 and text[i][0]!='<' and text[i][1]!='<':
        terms.append(tmp)
    i = i + 1
# print(terms)
print("数据已转化为列表")

# 将数据转化为条目数组
item_id = 0
items = [item()]
row = 0
tmp = ""
scale = 50
start = time.perf_counter()
while(row<len(terms)):

    i = int(row/len(terms))
    if(i>scale):
        i = scale
    a = "*" * i
    b = "." * (scale - i)
    c = (i / scale) * 100
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c,a,b,dur),end = "")

    # print(row)
    if(re.match(r"\d+\.\[期刊\]", terms[row]) != None):
        if(row!=0):
            item_id = item_id + 1
            items.append(item())
            row = row + 1
            continue
        else:
            row = row + 1
            continue
    if(re.match(r"题名:", terms[row]) != None):
        items[item_id].title = terms[row][3:]
        row = row + 1
        continue
    if(re.match(r"作者：", terms[row]) != None):
        tmp = terms[row][3:]
        row = row + 1
        while(True):
            if(re.match(r"作者单位：", terms[row]) == None and re.match(r"期刊名称：", terms[row]) == None):
                tmp = tmp + terms[row]
                row = row + 1
                continue
            else:
                tmp = tmp.split('（')
                tmp = tmp[0]
                items[item_id].authers = tmp
                break
        continue
    if(re.match(r"作者单位：", terms[row]) != None):
        items[item_id].units = terms[row][5:]
        row = row + 1
        continue
    if(re.match(r"期刊名称：", terms[row]) != None):
        items[item_id].journal = terms[row][5:]
        row = row + 1
        continue
    if(re.match(r"关键词：", terms[row]) != None):
        items[item_id].keywords = terms[row][4:]
        row = row + 1
        continue
    if(re.match(r"出处：", terms[row]) != None):
        items[item_id].source = terms[row][3:]
        row = row + 1
        continue
    if(re.match(r"摘要：", terms[row]) != None):
        items[item_id].abstract = terms[row][3:]
        row = row + 1
        continue
    if(re.match(r"链接地址：", terms[row]) != None):
        items[item_id].href = terms[row][5:]
        row = row + 1
        continue
    if(re.match(r"引证：", terms[row]) != None):
        items[item_id].quote = int(terms[row][3:])
        row = row + 1
        continue
    if(re.match(r"ISSN:", terms[row]) != None):
        items[item_id].ISSN = terms[row][5:]
        row = row + 1
        continue
print("数据已转化为条目列表")

数据已获取
数据已转化为列表
 0 %[->..................................................]0.01s数据已转化为条目列表


In [7]:
# 连接数据库
# try:
db = pymysql.connect(host='localhost', user='root', passwd='lzd18020226', port=3306, db='tysql')
print('连接成功！')
# except:
#     print('something wrong!')

# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 execute()  方法执行 SQL 查询
cursor.execute("SELECT VERSION()")
# 使用 fetchone() 方法获取单条数据.
data = cursor.fetchone()
print("Database version : %s " % data)

连接成功！
Database version : 8.0.34 


In [ ]:
# 建表
cursor.execute("DROP TABLE IF EXISTS PAPERS")

# 使用预处理语句创建表
sql = """CREATE TABLE PAPERS (
         TITLE  CHAR(100) NOT NULL,
         AUTHERS CHAR(100),
         UNITS CHAR(150),  
         JOURNAL CHAR(30),
         KEYWORDS CHAR(100),
         SOURCE CHAR(200),
         ABSTRACT CHAR(1000),
         HREF CHAR(150),
         QUOTE INT,
         ISSN CHAR(10) )
         character set utf8mb4 collate utf8mb4_general_ci;"""

cursor.execute(sql)
print('建表成功！')

In [ ]:
def items_to_MySQL(items):
    for item in items:
        # 插入
        sql = """INSERT INTO PAPERS(TITLE,
                AUTHERS, UNITS, JOURNAL, KEYWORDS, SOURCE, ABSTRACT, HREF, QUOTE, ISSN)
                VALUES (""" \
                + item.title + ", " \
                + item.authers + ", " \
                + item.units + ", " \
                + item.journal + ", " \
                + item.keywords + ", " \
                + item.source + ", " \
                + item.abstract + ", " \
                + item.href + ", " \
                + str(item.quote) + ", " \
                + item.ISSN + ")"
        try:
            # 执行sql语句
            cursor.execute(sql)
            # 提交到数据库执行
            db.commit()
            print('数据插入成功！')
        except:
            # 如果发生错误则回滚
            db.rollback()
            print('数据插入错误！')

items_to_MySQL(items)